## Network Validation Checks

#### Import Libraries 

In [ ]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
# from nxviz import CircosPlot
from shapely.geometry import LineString

#### Load Network data from Shapefiles

In [ ]:
def segment_heading(line, first_segment=True, flip=False):
    if flip:
        line = LineString(line.coords[::-1])
    
    segment = line.coords[:2] if first_segment else line.coords[-2:]
    
    from_pt = segment[0]
    to_pt = segment[1]
    
    y_delta = to_pt[1] - from_pt[1]
    x_delta = to_pt[0] - from_pt[0]

    angle_radians = np.arctan2(y_delta, x_delta)
    if angle_radians < 0:
        angle_radians = 2 * np.pi + angle_radians;

    # Deal with the angles being from East Counterclockwise to Northerly bearings
    degrees = 90 - angle_radians * 180 / np.pi
    if degrees < 0:
        degrees = degrees + 360

    return degrees

In [ ]:
line_shp = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master.shp')).to_crs(epsg=2274)
node_shp = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master_Node.shp')).to_crs(epsg=2274)

# Heading AB
line_shp['AB_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True)
line_shp['AB_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False)
# Heading BA
line_shp['BA_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True, flip=True)
line_shp['BA_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False, flip=True)

### Build Network Graph

#### Turn Two-Way Links into 2 One-Way Links for Directional Flow.
#### Create Function: *directional_links*
Keep AB Links and produce BA Links, carrying over link attributes based on the direction of flow.

In [ ]:
def directional_links(atts, line_shp):
    dirs = {'keep':'AB', 'invert':'BA'}
    dfs = {}
    
    for flow,d in dirs.items():
        links = line_shp[line_shp['{}_LANES'.format(d)] > 0].copy()
        rename_dict = {'{}{}'.format(d, a):a for a in atts}
        links = links.rename(columns=rename_dict)
        
        if flow == 'invert':
            links['A'] = links['TO_ID']
            links['B'] = links['FROM_ID']
            links = links.drop(columns=['FROM_ID', 'TO_ID']).rename(columns={'A': 'FROM_ID', 'B': 'TO_ID'})
        
        dfs['{}_links'.format(d)] = links
    
    links = pd.concat(dfs.values(), ignore_index=True)
    
    drop_dict = ['{}{}'.format(d, a) for a in atts for d in dirs.values()]
    links = links.drop(columns=drop_dict)
    
    return links

#### Generate Directional links with Attributes
####  Call Function: *directional_links*
Use list of Directional Attributes to carry over, e.g. 'AB_LANES' and 'BA_LANES', to '_LANES' 

In [ ]:
# List of Directional Attributes
atts = [
    '_LINKDIR',
    '_LINKDTR',
    '_LANES',
    '_PARKING',
    '_TRIMS',
    '_AADT',
    '_CAR_ADT',
    '_SUT_ADT',
    '_MUT_ADT',
    '_BASEVOL',
    '_AFFTIME',
    '_AFFSPD',
    '_UCDELAY',
    '_DLYCAP',
    '_AMCAP',
    '_PMCAP',
    '_BPRA',
    '_BPRB',
    '_START_HEADING',
    '_END_HEADING'
]
# Generate Directional Links
line_shp = directional_links(atts, line_shp)

####  Build the Directional Graph: *g*
#### Call Method: *nx.from_pandas_edgelist*

In [ ]:
g = nx.from_pandas_edgelist(
    line_shp,
    source='FROM_ID',
    target='TO_ID',
    edge_attr=True,
    create_using=nx.DiGraph()
)

assert nx.is_directed(g)

### Link to Link validation Functions

#### Create Function: *link_matcher_atts*: 
For each node in Network, match all combinations of In/ Out Links and return both In and Out values

In [ ]:
def link_matcher_atts(attributes, graph, edge_id='ID'):
    heading = True if '_HEADING' in attributes else False
    atts = []
    atts.extend([a for a in attributes if a!='_HEADING'])
    
#     if heading: atts.remove('_HEADING')

    node_feed = []

    for node in node_shp['ID']:
        if node not in g:
            print('Node not in network: {}'.format(node))
            continue

        for i_o, i_d, i_data  in g.in_edges(node, data=True):
            for o_o, o_d, o_data in g.out_edges(node, data=True):
                if i_o == o_d and i_d == o_o:
                    # This is just a U-Turn on the "same" link
                    continue
                    
                data = [i_data, o_data]
                values = [node, *[d[edge_id] for d in data], *[d[a] for a in atts for d in data]]
                if heading: values.extend([i_data['_END_HEADING'], o_data['_START_HEADING']])
                node_feed.append(values)
                
    # Generate Column names
    dirs = ['in', 'out']
    cols = ['node', *[ d+'_'+edge_id for d in dirs], *[ d+'_'+a for a in atts for d in dirs]]
    if heading: cols.extend([ d+'__HEADING' for d in dirs])

    return pd.DataFrame(node_feed, columns=cols)

#### Create Function: *attribute_filter_atts*: 
Apply Attribute to filter by difference value 

In [ ]:
def attribute_change(network, attributes):
    for att in attributes:
        in_fld = 'in_{}'.format(att)
        out_fld = 'out_{}'.format(att)
        fld_diff = 'diff_{}'.format(att)
#         abs_fld_diff = 'abs_diff_{}'.format(filter_name)
    
        if isinstance(network[in_fld][0], (int, float)):
            network[fld_diff] = network[out_fld] - network[in_fld]
        elif isinstance(network[in_fld][0], str):
            network[fld_diff] = 'SAME' if network[out_fld].str == network[in_fld].str else 'CHANGES'

    return network#.groupby([in_fld, out_fld]).count().reset_index()

In [ ]:
def attribute_filtering(network, att_filters, get_node_max):
    for filter_name, filter_val in att_filters.items():
        in_fld = 'in_{}'.format(filter_name)
        out_fld = 'out_{}'.format(filter_name)
        fld_diff = 'diff_{}'.format(filter_name)
#         abs_fld_diff = 'abs_diff_{}'.format(filter_name)
    
        if isinstance(filter_val, (int, float)):
            network = network[abs(network[fld_diff]) >= filter_val]
        elif isinstance(filter_val, str):
            network = network[(network[in_fld] == filter_val) | (network[out_fld] == filter_val)]

    return network#.groupby([in_fld, out_fld]).count().reset_index()

In [ ]:
# In/Out Long Table
def export_geo_compare(network, att_name, att_name_add=None):
    fld_diff = 'diff_{}'.format(att_name)
    
    network = network.rename(columns={'out_ID':'ID'})

    # geometries
    geo_cols = ['ID', 'geometry']
#     if att_name_add: geo_cols.append(att_name_add)
    network = pd.merge(line_shp[geo_cols], network, on='ID')
    
    shp_export = os.path.join('..', 'data', 'Network_attribute_change_{}.shp'.format('_'.join([n for n in att_name])))
    network.to_file(shp_export)
#     return network#.sort_values(by=fld_diff, ascending=False), shp_export
    return network, shp_export

#### Create Function: *link_compare*: 
Call functions to calculate Link-to-Link attribute comparison

In [ ]:
def link_compare(attribute_names, attribute_filters, get_node_max):
    network = link_matcher_atts(attribute_names, graph=g)
    network = attribute_change(network, attribute_names)
    network = attribute_filtering(network,
                                attribute_filters,
                                get_node_max
                                )
    csv_export = os.path.join('..', 'data', 'Network_attribute_change_table_{}.csv'.format('_'.join([n for n in attribute_names])))
    network.to_csv(csv_export, index=False)
    
    # Print
    print_txt = 'Total {} network links with change in {}: {:,}'
    print(print_txt.format('SUMMARIZED' if get_node_max else '\b',
                           ', '.join([n for n in attribute_names]),
                           len(network)
                           ))
    print('Table exported to {}'.format(csv_export))
    if get_node_max:
        network, shp_export = export_geo_compare(network, attribute_filters)
        print('Shapefile exported to {}'.format(shp_export))

    return network

## VALIDATION TESTS

### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and FUNCTIONAL CLASS

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', '_LANES', '_HEADING']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
    '_LANES':2,
    '_HEADING':70
}
# Calculate Heading deviation of In-Link(Final Segment) and Out-Link (Initial Segment)

# Summarize by Node Maximum value and Export Geo:
get_node_max = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, get_node_max)
network.head()

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', '_LANES']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
    '_LANES':2
}
# Summarize by Node Maximum value and Export Geo:
get_node_max = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, get_node_max)
network.head()

### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and FUNCTIONAL CLASS

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', '_LANES']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
    '_LANES':2
}
# Summarize by Node Maximum value and Export Geo:
get_node_max = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, get_node_max)
network.head()

### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and ROUTE NAME
Filter by Speed Limit

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', 'RTE_NME']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
}
# Summarize by Node Maximum value and Export Geo:
get_node_max = False


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, get_node_max)
network.head()

### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and ROUTE NAME
Filter by Route Name

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', 'RTE_NME']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
    'RTE_NME':'CENTROID CONNECTOR',
}
# Summarize by Node Maximum value and Export Geo:
get_node_max = False


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, get_node_max)
network.head()

### Network Link-to-Link attribute side-by-side validation: NUMBER OF LANES and FUNCTIONAL CLASS
Filter by Number of Lanes

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['_LANES', 'FUNCCLASS']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    '_LANES':2,
}
# Summarize by Node Maximum value and Export Geo:
get_node_max = False


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, get_node_max)
network.head()

### Put attributes on the network

In [ ]:
node_shp['X'] = node_shp.geometry.x
node_shp['Y'] = node_shp.geometry.y
nx.set_node_attributes(g, pd.Series(node_shp.X, index=node_shp.ID).to_dict(), 'X')
nx.set_node_attributes(g, pd.Series(node_shp.Y, index=node_shp.ID).to_dict(), 'Y')

### Visualizations (Slow- Due to Network Size)

In [ ]:
# plt.subplot(121)

#nx.draw(g, with_labels=True, font_weight='bold')
#plt.subplot(122)

#nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(g, nx.spring_layout(g), node_size=10, node_color='blue')
plt.show()
